# Descargar Tweet con Tweepy

In [2]:
#Importamos librerias
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import tweepy
import json
import pandas as pd
import csv
import re
import sys
from textblob import TextBlob
import string
import preprocessor as p
import os
import time


## Llamada a la API y Descarga de Tweets

In [7]:
# Credenciasles Twitter
# Se obtienen desde tu cuentas de  twitter developer.
consumer_key ='Rellenar con tus credenciales'
consumer_secret ='Rellenar con tus credenciales'
access_key ='Rellenar con tus credenciales'
access_secret ='Rellenar con tus credenciales'

# Pasa tus credenciales de  twitter a tweepy via OAuthHandler.
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth, wait_on_rate_limit = True)

In [8]:
#Función para descargarse los tweet.

def text_query_to_csv(text_query,count,data_since):
    try:
        all_tweets = []
        
        # Creacción de  queryusando los parametos que necesitamos
        print("Descargando Tweets")
        tweets = tweepy.Cursor(api.search,q=text_query,lang = 'es', locale ='UTF-8',since= data_since, tweet_mode='extended').items(count)
        all_tweets = all_tweets + list(tweets)
        print("Descargados Tweets")
        #Finalmente no necesira time.sleep,lo hace automaticamente poniendo un parametro en la función API.
        #print("Esperando 15 min")
        # esperar 15 min api limit
        #time.sleep(60*15)
            
        # Iteramos para sacar la información que queremos de cada tweet.
        tweets_list = [[tweet.created_at, tweet.id,tweet.user.created_at,tweet.user.screen_name,tweet.user.description,tweet.user.location,
                        tweet.user.friends_count,tweet.user.followers_count,tweet.user.statuses_count,tweet.retweet_count,
                        tweet.full_text,tweet.entities['hashtags']] for tweet in all_tweets]

        # Creaamos DataFrame con la lista de tweets
        tweets_df = pd.DataFrame(tweets_list,columns=['Datetime','Tweet Id','Creacion','Username','Description','Location',
                                                      'Following','Followers','Total Tweets','RT','Text','Hashtags'])

        #Convertimos Df en csv
        tweets_df.to_csv('{}-tweets.csv'.format('#LaIslaDeLasTentaciones5'), sep=',', index = False)

    except BaseException as e:
        print('failed on_status,',str(e))
        #time.sleep(3)

In [13]:
# Valores con los que vamos a realizar la busqueda.

#Hashtag que vamos a buscar y filtros que aplicamos en la busqueda. 
text_query = '(#LaIslaDeLasTentaciones5) -filter:links -filter:replies -filter:retweets'
#Numero de tweets.
count = 200000
#Fecha desde que empezamos a buscar
data_since ='2021-02-11'


# Llamamos a la función 
text_query_to_csv(text_query,count,data_since)

Descargando Tweets
Descargados Tweets


## Limpieza Tweets

In [8]:
#Importamos DF creado.
df =pd.read_excel('#LaIslaDeLasTentaciones5-tweets.xlsx')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66744 entries, 0 to 66743
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    66744 non-null  int64 
 1   Datetime      66744 non-null  object
 2   Tweet Id      66744 non-null  int64 
 3   Creacion      66744 non-null  object
 4   Username      66744 non-null  object
 5   Description   57715 non-null  object
 6   Location      40526 non-null  object
 7   Following     66744 non-null  int64 
 8   Followers     66744 non-null  int64 
 9   Total Tweets  66744 non-null  int64 
 10  RT            66744 non-null  int64 
 11  Text          66744 non-null  object
 12  Hashtags      66744 non-null  object
dtypes: int64(6), object(7)
memory usage: 6.6+ MB


In [11]:
df.tail()

,Unnamed: 0,Datetime,Tweet Id,Creacion,Username,Description,Location,Following,Followers,Total Tweets,RT,Text,Hashtags
66739,66864,2021-02-11 21:20:02,1359975507085910016,2020-10-12 21:57:55,90srosinkranz,"thank u, next — track 5",she¡!her,421,419,2855,0,ay lucia mi amor lo que vas a llorar yo me enc...,"[{'text': 'LaIslaDeLasTentaciones5', 'indices'..."
66740,66865,2021-02-11 21:20:02,1359975506393918976,2019-05-06 13:25:17,EscandaloNando,"Eurofan, adicto a los realitys y fan de Pokemo...","Badajoz, España",1747,2102,10826,0,"De las 3 ediciones, es la que menos me está gu...","[{'text': 'LaIslaDeLasTentaciones5', 'indices'..."
66741,66866,2021-02-11 21:20:01,1359975502639955968,2016-02-07 13:23:26,mariiamc02,"Si me quieres, dame palomitas",España,321,392,38655,1,Todos somos Lara. Todos queremos tocar el pelo...,"[{'text': 'LaIslaDeLasTentaciones5', 'indices'..."
66742,66867,2021-02-11 21:20:00,1359975497237691904,2018-08-22 20:58:39,whateverrcloud,NaN,NaN,347,304,2940,1,uf qué guapa lara tentación #LaIslaDeLasTenta...,"[{'text': 'LaIslaDeLasTentaciones5', 'indices'..."
66743,66868,2021-02-11 21:20:00,1359975496231058944,2012-12-22 23:30:21,Alba1169,Jugadora del VEC Barcelona,NaN,136,251,2371,0,He sido capaz de no tragarme ningún spoiler po...,"[{'text': 'LaIslaDeLasTentaciones5', 'indices'..."


### Limpieza general 

In [12]:
# Función para realizar limpieza general.

def cleanUpTweet(txt):
    # Remove mentions
    txt = re.sub(r'@[A-Za-z0-9_]+', '', txt)
    # Remove hashtags
    txt = re.sub(r'#', '', txt)
    # Remove retweets:
    txt = re.sub(r'RT : ', '', txt)
    # Remove urls
    txt = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', ' ', txt)
    #Remove line break
    txt = re.sub(r'\n', ' ', txt )
    #Remove-
    txt = re.sub(r'-', ' ', txt )
    return txt


In [13]:
#Aplicamos función

df['Text'] = df['Text'].apply(cleanUpTweet)


In [14]:
df.tail(5)

,Unnamed: 0,Datetime,Tweet Id,Creacion,Username,Description,Location,Following,Followers,Total Tweets,RT,Text,Hashtags
66739,66864,2021-02-11 21:20:02,1359975507085910016,2020-10-12 21:57:55,90srosinkranz,"thank u, next — track 5",she¡!her,421,419,2855,0,ay lucia mi amor lo que vas a llorar yo me enc...,"[{'text': 'LaIslaDeLasTentaciones5', 'indices'..."
66740,66865,2021-02-11 21:20:02,1359975506393918976,2019-05-06 13:25:17,EscandaloNando,"Eurofan, adicto a los realitys y fan de Pokemo...","Badajoz, España",1747,2102,10826,0,"De las 3 ediciones, es la que menos me está gu...","[{'text': 'LaIslaDeLasTentaciones5', 'indices'..."
66741,66866,2021-02-11 21:20:01,1359975502639955968,2016-02-07 13:23:26,mariiamc02,"Si me quieres, dame palomitas",España,321,392,38655,1,Todos somos Lara. Todos queremos tocar el pelo...,"[{'text': 'LaIslaDeLasTentaciones5', 'indices'..."
66742,66867,2021-02-11 21:20:00,1359975497237691904,2018-08-22 20:58:39,whateverrcloud,NaN,NaN,347,304,2940,1,uf qué guapa lara tentación LaIslaDeLasTentac...,"[{'text': 'LaIslaDeLasTentaciones5', 'indices'..."
66743,66868,2021-02-11 21:20:00,1359975496231058944,2012-12-22 23:30:21,Alba1169,Jugadora del VEC Barcelona,NaN,136,251,2371,0,He sido capaz de no tragarme ningún spoiler po...,"[{'text': 'LaIslaDeLasTentaciones5', 'indices'..."


### Limpieza abreviaturas, reemplazar palabras y pasar a minuscula

In [15]:
#Abreviaturas 
abbrevs = {'d': 'de',
               'x': 'por',
               'xa': 'para',
               'as': 'has',
               'q': 'que',
               'ke': 'que',
               'k': 'que',
               'dl': 'del',
               'xq': 'porqué',
               'dr': 'doctor',
               'dra': 'doctora',
               'sr': 'señor',
               'sra': 'señora',
               'm': 'me'}
elementos = abbrevs.items()

In [16]:
#Función para remplazar abreviaturas, Hashtag y transformar texto en minuscula

re.sub(r"\bis\b", "was", "This is the best island!")
lista_abreviaturas =[]
dfMo = df.copy()
for a, b in elementos:
    #lista_abreviaturas = list(map(lambda x : x.replace(x,i), df['Text']))
    a = " " +a + " "
    b = " " +b+ " "
    
    #dfMo['Text'] = dfMo['Text'].apply(lambda x :re.sub(a, b, x))
    dfMo['Text'] = dfMo['Text'].str.replace(a,b,regex =True)

#Hashtag    

dfMo['Text'] = dfMo['Text'].apply(lambda x : x.replace('LaIslaDeLasTentaciones5','La Isla'))

#Minusculas

dfMo['Text']  = dfMo['Text'].apply(lambda x :x.lower())

print(dfMo['Text'])

    
    

0        es que no me creo que sea la tercera temporada...
1        marina: ¿y en que clase de juegos os coméis la...
2        vacunilla president!!! otegui lendakari !! sán...
3            hugo me deja sin palabras, en serio.  la isla
4        lucia va a rerollear a dps desde tank. desde t...
                               ...                        
66739    ay lucia mi amor lo que vas a llorar yo me enc...
66740    de las 3 ediciones, es la que menos me está gu...
66741    todos somos lara. todos queremos tocar el pelo...
66742                 uf qué guapa lara tentación  la isla
66743    he sido capaz de no tragarme ningún spoiler po...
Name: Text, Length: 66744, dtype: object


### Limpieza emojis

In [22]:
#Función para eliminar emojis
def remove_emoji(string):
    emoji_pattern = re.compile("["
                          u"\U0001F600-\U0001F64F"  # emoticonos
                               u"\U0001F300-\U0001F5FF"  # simbolos y pictogramas
                               u"\U0001F680-\U0001F6FF"  # Trasporte y s
                               u"\U0001F1E0-\U0001F1FF"  # Banderas (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  
                               u"\u3030"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)



In [23]:
#Aplicamos función

dfMo['Text'] = dfMo['Text'].apply(remove_emoji)

In [32]:
#Guardamos DF con los tweets limpios

dfMo.to_csv('LaIslaDeLastenataciones5-tweets-limpio-completo.csv')

In [31]:
dfMo.to_excel('LaIslaDeLastenataciones5-tweets-limpio-completo.xlsx')